In [50]:
import cx_Oracle
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = con.cursor()
sql = '''SELECT lcl2.고객번호, 제휴사, 세분류명, count(구매금액) 구매빈도, sum(구매금액) 구매금액 from lcl2
join 군집 on lcl2.고객번호 = 군집.고객번호
where 군집=1
group by lcl2.고객번호, 제휴사, 세분류명
order by lcl2.고객번호'''
cursor.execute(sql)
x = cursor.fetchall()
columns = ['고객번호', '제휴사', '세분류명', '구매빈도', '구매금액']
df = pd.DataFrame(x, columns = columns)
cursor.close()
con.commit()
con.close()

df_shop, df_conv, df_spec=get_indexing(df)
df1 = pd.concat([df_shop, df_conv, df_spec])

In [51]:
con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = con.cursor()
sql = '''SELECT lcl2.고객번호, 제휴사, 세분류명,분류, count(구매금액) 구매빈도, sum(구매금액) 구매금액 from lcl2
join 군집 on lcl2.고객번호 = 군집.고객번호
where 군집=1
group by lcl2.고객번호, 제휴사, 세분류명,분류
order by lcl2.고객번호'''
cursor.execute(sql)
x = cursor.fetchall()
columns = ['고객번호', '제휴사', '세분류명','분류', '구매빈도', '구매금액']
df = pd.DataFrame(x, columns = columns)
cursor.close()
con.commit()
con.close()

df_shop, df_conv, df_spec=get_indexing(df)
df2 = pd.concat([df_shop, df_conv, df_spec])

In [52]:
con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = con.cursor()
sql = '''SELECT lcl2.고객번호, 제휴사, 세분류명,분류,분류등급, count(구매금액) 구매빈도, sum(구매금액) 구매금액 from lcl2
join 군집 on lcl2.고객번호 = 군집.고객번호
where 군집=1
group by lcl2.고객번호, 제휴사, 세분류명,분류,분류등급
order by lcl2.고객번호'''
cursor.execute(sql)
x = cursor.fetchall()
columns = ['고객번호', '제휴사', '세분류명','분류','분류등급', '구매빈도', '구매금액']
df = pd.DataFrame(x, columns = columns)
cursor.close()
con.commit()
con.close()

df_shop, df_conv, df_spec=get_indexing(df)
df3 = pd.concat([df_shop, df_conv, df_spec])

In [53]:
con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = con.cursor()
sql = '''SELECT lcl2.고객번호, 제휴사, 세분류명,분류,분류등급,소분류명,count(구매금액) 구매빈도, sum(구매금액) 구매금액 from lcl2
join 군집 on lcl2.고객번호 = 군집.고객번호
where 군집=1
group by lcl2.고객번호, 제휴사, 세분류명,분류,분류등급,소분류명
order by lcl2.고객번호'''
cursor.execute(sql)
x = cursor.fetchall()
columns = ['고객번호', '제휴사', '세분류명','분류','분류등급','소분류명' ,'구매빈도', '구매금액']
df = pd.DataFrame(x, columns = columns)
cursor.close()
con.commit()
con.close()

df_shop, df_conv, df_spec=get_indexing(df)
df4 = pd.concat([df_shop, df_conv, df_spec])

In [54]:
def get_ui_df(df):
    df1 = df.copy()

    df1_child = df1.loc[df1['분류'] == '유아동']
    df1_processed_food = df1.loc[df1['분류'] == '가공식품']
    df1_fresh_food= df1.loc[df1['분류'] == '신선식품']
    df1_medicine = df1.loc[df1['분류'] == '의약품/의료기기']
    df1_etc = df1.loc[df1['분류'] == '기타']
    df1_daily_necessities = df1.loc[df1['분류'] == '일상용품']
    df1_restaurant = df1.loc[df1['분류'] == '외식']
    df1_clothing = df1.loc[df1['분류'] == '의류']
    df1_furniture = df1.loc[df1['분류'] == '가구/인테리어']
    df1_digital = df1.loc[df1['분류'] == '디지털/가전']
    df1_fashion = df1.loc[df1['분류'] == '패션잡화']
    df1_luxury = df1.loc[df1['분류'] == '명품']
    df1_leisure = df1.loc[df1['분류'] == '전문스포츠/레저']
    df1_education = df1.loc[df1['분류'] == '교육/문화용품']
    
    return df1_child, df1_processed_food, df1_fresh_food, df1_medicine,df1_etc,df1_daily_necessities,df1_restaurant,df1_clothing,df1_furniture,\
    df1_digital,df1_fashion,df1_luxury,df1_leisure,df1_education

# 세카테고리로 나눠 지수화하기
def get_indexing(df):
    df1 = df.copy()
    try:
        df_shop = pd.DataFrame()
        df_shop = df1.loc[df['세분류명'] == '선매품']
        df_shop['구매지수'] = (MinMaxScaler(feature_range = (1, 100)).fit_transform(df_shop[['구매빈도']]) * 0.14) + (MinMaxScaler(feature_range = (1, 100)).fit_transform(df_shop[['구매금액']]) * 0.86)
        df_conv = pd.DataFrame()
        df_conv = df1.loc[df['세분류명'] == '편의품']
        df_conv['구매지수'] = (MinMaxScaler(feature_range = (1, 100)).fit_transform(df_conv[['구매빈도']]) * 0.7) + (MinMaxScaler(feature_range = (1, 100)).fit_transform(df_conv[['구매금액']]) * 0.3)
        df_spec = pd.DataFrame()
        df_spec = df1.loc[df['세분류명'] == '전문품']
        df_spec['구매지수'] = (MinMaxScaler(feature_range = (1, 100)).fit_transform(df_spec[['구매빈도']]) * 0.025) + (MinMaxScaler(feature_range = (1, 100)).fit_transform(df_spec[['구매금액']]) * 0.975)
    except:
        pass
    return df_shop, df_conv, df_spec

In [56]:
import surprise
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.dataset import DatasetAutoFolds

In [57]:
def get_algo(df,target):
    # 소분류명 정보 데이터 프레임
    items=pd.DataFrame(data=df[target].unique(),columns=[target])
    items.head()
    reader = Reader(rating_scale=(0, 100))
    data = Dataset.load_from_df(df[['고객번호',target,'구매지수']],reader)
    trainset = data.build_full_trainset()
    algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
    algo.fit(trainset)
    return algo,items

def recomm_item_by_surprise(algo, userId, top_n,items,target): # userId 당 
    predictions = [algo.predict(str(userId),str(item)) for item in items[target]]
    def sortkey_est(pred):
        return pred.est 
    predictions.sort(key = sortkey_est, reverse = True)  # 예측 확률만큼 내림차순으로 정렬   
    top_predictions = predictions[:top_n] 
    return top_predictions

In [58]:
result_df=pd.DataFrame(columns=['고객번호','소비재','카테고리','등급'])
custom_list=[]
goods_list=[]
for custom in df1.고객번호.unique():
    algo, items = get_algo(df1,'세분류명')
    top_item_preds = recomm_item_by_surprise(algo, custom, 1 ,items,'세분류명')
    custom_list.append(top_item_preds[0][0])
    goods_list.append(top_item_preds[0][1])

result_df['고객번호']=custom_list
result_df['소비재']=goods_list

In [59]:
Specialty = ['명품']
Shopping = ['가공식품','신선식품','외식','일상용품','의약품/의료기기','교육/문화용품']
Convenience = ['디지털/가전','가구/인테리어','의류','전문스포츠/레저','패션잡화','유아동']

In [60]:
idx = df2[df2['분류'] == '기타'].index
df2 = df2.drop(idx)

In [61]:
df2_temp=pd.DataFrame(columns=df2.columns)

for i,j in zip(result_df.고객번호.unique(),result_df.소비재):
    df2_temp_=df2[(df2['고객번호']==i) & (df2['세분류명']==j)]
    df2_temp = pd.concat([df2_temp,df2_temp_])

In [62]:
df2_temp

,고객번호,제휴사,세분류명,분류,구매빈도,구매금액,구매지수
7,00007,A,편의품,가공식품,52,949780,2.266220
8,00007,A,편의품,신선식품,77,1393050,2.881128
9,00007,A,편의품,외식,210,2753810,5.893552
10,00007,A,편의품,일상용품,24,3121650,2.270099
15,00007,B,편의품,가공식품,67,417640,2.428014
...,...,...,...,...,...,...,...
21920,19325,B,편의품,신선식품,19,92578,1.383920
21921,19325,B,편의품,일상용품,23,211340,1.494722
21922,19325,C,편의품,가공식품,1,2400,1.000618
21923,19325,C,편의품,신선식품,2,8101,1.022092


In [63]:
clf_list=[]
custom_list=[]
for custom in df2_temp.고객번호.unique():
    df2_temp_=df2_temp[df2_temp.고객번호==custom]
    algo, items = get_algo(df2_temp_,'분류')
    top_item_preds = recomm_item_by_surprise(algo, custom, 1 ,items,'분류')
    clf_list.append(top_item_preds[0][1])
    custom_list.append(top_item_preds[0][0])

In [64]:
clf=pd.DataFrame(columns=['고객번호','카테고리'])
clf['고객번호']=custom_list
clf['카테고리']=clf_list

In [65]:
clf

,고객번호,카테고리
0,00007,외식
1,00014,신선식품
2,00017,신선식품
3,00029,의류
4,00045,의류
...,...,...
1089,19178,신선식품
1090,19183,신선식품
1091,19205,외식
1092,19268,가공식품


In [66]:
result_df_merge = pd.merge(result_df,clf, how='outer',on='고객번호')

In [67]:
result_df_merge=result_df_merge.drop(['카테고리_x','등급'],axis=1)

In [68]:
result_df_merge.columns=['고객번호','소비재','카테고리']

In [69]:
result_df_merge

,고객번호,소비재,카테고리
0,00007,편의품,외식
1,00014,편의품,신선식품
2,00017,편의품,신선식품
3,00029,선매품,의류
4,00045,선매품,의류
...,...,...,...
1089,19178,편의품,신선식품
1090,19183,편의품,신선식품
1091,19205,편의품,외식
1092,19268,편의품,가공식품


In [70]:
df3_temp=pd.DataFrame(columns=df3.columns)

for i,j,k in zip(result_df_merge.고객번호.unique(),result_df_merge.소비재,result_df_merge.카테고리):
    df3_temp_=df3[(df3['고객번호']==i) & (df3['세분류명']==j) & (df3['분류']==k)]
    df3_temp = pd.concat([df3_temp,df3_temp_])

In [71]:
clf_list=[]
custom_list=[]
for custom in df3_temp.고객번호.unique():
    df3_temp_=df3_temp[df3_temp.고객번호==custom]
    algo, items = get_algo(df3_temp_,'분류등급')
    top_item_preds = recomm_item_by_surprise(algo, custom, 1 ,items,'분류등급')
    clf_list.append(top_item_preds[0][1])
    custom_list.append(top_item_preds[0][0])

In [72]:
clf=pd.DataFrame(columns=['고객번호','분류등급'])
clf['고객번호']=custom_list
clf['분류등급']=clf_list

In [73]:
result_df_final = pd.merge(result_df_merge,clf, how='outer',on='고객번호')

In [74]:
df4

,고객번호,제휴사,세분류명,분류,분류등급,소분류명,구매빈도,구매금액,구매지수
0,00007,A,선매품,가구/인테리어,가구/인테리어_상,식탁,2,750000,1.076580
1,00007,A,선매품,가구/인테리어,가구/인테리어_중,N.B,1,98000,1.004931
2,00007,A,선매품,가구/인테리어,가구/인테리어_중,한실침구,2,86500,1.043176
3,00007,A,선매품,가구/인테리어,가구/인테리어_하,수예소품,2,84600,1.043080
4,00007,A,선매품,디지털/가전,디지털/가전_하,수입소형가전,1,37800,1.001901
...,...,...,...,...,...,...,...,...,...
293914,19205,A,전문품,명품,명품_하,직수입,2,254820,1.158269
293989,19268,A,전문품,명품,명품_하,어덜트,3,1906400,1.926531
294147,19325,A,전문품,명품,명품_중,명품잡화,3,488000,1.308414
294148,19325,A,전문품,명품,명품_중,수입의류,6,1191000,1.760359


In [75]:
df4_temp=pd.DataFrame(columns=df4.columns)

for i,j,k,l in zip(result_df_final.고객번호.unique(),result_df_final.소비재,result_df_final.카테고리,result_df_final.분류등급):
    df4_temp_=df4[(df4['고객번호']==i) & (df4['세분류명']==j) & (df4['분류']==k) & (df4['분류등급']==l)]
    df4_temp = pd.concat([df4_temp,df4_temp_])

In [76]:
df4_temp

,고객번호,제휴사,세분류명,분류,분류등급,소분류명,구매빈도,구매금액,구매지수
76,00007,A,편의품,외식,외식_중,디저트카페,5,81000,1.910111
77,00007,A,편의품,외식,외식_중,떡,1,10000,1.003021
78,00007,A,편의품,외식,외식_중,면류,4,99000,1.694147
79,00007,A,편의품,외식,외식_중,밥류,12,127900,3.474131
80,00007,A,편의품,외식,외식_중,브랑제리,32,374790,7.976895
...,...,...,...,...,...,...,...,...,...
294174,19325,A,편의품,외식,외식_중,초밥,4,158500,1.712137
294175,19325,A,편의품,외식,외식_중,커피숍,10,67150,3.012952
294176,19325,A,편의품,외식,외식_중,케익류,2,12500,1.225182
294177,19325,A,편의품,외식,외식_중,패밀리레스토랑,2,67600,1.241842


In [77]:
clf_list=[]
custom_list=[]
for custom in df4_temp.고객번호.unique():
    df4_temp_=df4_temp[df4_temp.고객번호==custom]
    algo, items = get_algo(df4_temp_,'소분류명')
    top_item_preds = recomm_item_by_surprise(algo, custom, 1 ,items,'소분류명')
    for item in top_item_preds:
        clf_list.append(item[1])
        custom_list.append(item[0])

In [78]:
clf=pd.DataFrame(columns=['고객번호','분류등급'])
clf['고객번호']=custom_list
clf['분류등급']=clf_list

In [79]:
result_df_final

,고객번호,소비재,카테고리,분류등급
0,00007,편의품,외식,외식_중
1,00014,편의품,신선식품,신선식품_중
2,00017,편의품,신선식품,신선식품_상
3,00029,선매품,의류,의류_중
4,00045,선매품,의류,의류_상
...,...,...,...,...
1089,19178,편의품,신선식품,신선식품_하
1090,19183,편의품,신선식품,신선식품_중
1091,19205,편의품,외식,외식_중
1092,19268,편의품,가공식품,가공식품_중


In [80]:
clf

,고객번호,분류등급
0,00007,한식델리
1,00014,청과
2,00017,우육
3,00029,L/C골프의류
4,00045,디자이너부틱
...,...,...
1089,19178,두부류
1090,19183,유기농채소
1091,19205,전문베이커리
1092,19268,커피음료


In [81]:
temp = pd.merge(result_df_final,clf, how='outer',on='고객번호')

In [82]:
temp.columns=['고객번호','소비재','카테고리','분류등급','제품']

In [87]:
temp.set_index(np.arange(1,1095))

,고객번호,소비재,카테고리,분류등급,제품
1,00007,편의품,외식,외식_중,한식델리
2,00014,편의품,신선식품,신선식품_중,청과
3,00017,편의품,신선식품,신선식품_상,우육
4,00029,선매품,의류,의류_중,L/C골프의류
5,00045,선매품,의류,의류_상,디자이너부틱
...,...,...,...,...,...
1090,19178,편의품,신선식품,신선식품_하,두부류
1091,19183,편의품,신선식품,신선식품_중,유기농채소
1092,19205,편의품,외식,외식_중,전문베이커리
1093,19268,편의품,가공식품,가공식품_중,커피음료


In [98]:
temp.value_counts('소비재')

소비재
편의품    683
선매품    254
전문품    157
dtype: int64

In [84]:
df=pd.pivot_table(temp, index = ['고객번호','소비재','카테고리','분류등급','제품'])

In [85]:
df

,,,,
고객번호,소비재,카테고리,분류등급,제품
00007,편의품,외식,외식_중,한식델리
00014,편의품,신선식품,신선식품_중,청과
00017,편의품,신선식품,신선식품_상,우육
00029,선매품,의류,의류_중,L/C골프의류
00045,선매품,의류,의류_상,디자이너부틱
...,...,...,...,...
19178,편의품,신선식품,신선식품_하,두부류
19183,편의품,신선식품,신선식품_중,유기농채소
19205,편의품,외식,외식_중,전문베이커리


In [36]:
result_df_final.to_csv('추천상품목록.csv')